Setup

In [6]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Chatbot

In [4]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    max_tokens=500,
    timeout=None,
    max_retries=1,

)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except KeyboardInterrupt:
        print("\nGoodbye!")
        break

Assistant: The world’s oceans are vast bodies of saltwater that cover about 71% of the Earth's surface. There are five major oceans, which are:

1. **Pacific Ocean**: The largest and deepest ocean, it extends from the Arctic Ocean in the north to the Southern Ocean in the south and is bordered by Asia and Australia on the west and the Americas on the east.

2. **Atlantic Ocean**: The second-largest ocean, it lies between the Americas to the west and Europe and Africa to the east. It is known for its distinct S-shaped basin.

3. **Indian Ocean**: The third-largest ocean, it is located mainly between Africa, Asia, Australia, and the Indian subcontinent. It is known for its warm waters and significant trade routes.

4. **Southern Ocean**: Also known as the Antarctic Ocean, it encircles Antarctica and is recognized for its unique ecosystem and role in global climate regulation.

5. **Arctic Ocean**: The smallest and shallowest ocean, it is located around the Arctic region, bordered by Nort